In [ ]:
from torch.utils.data import DataLoader, Dataset
import os
from PIL import Image
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.utils import make_grid, save_image
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import csv
from datetime import datetime

In [ ]:
!nvidia-smi

In [ ]:
class CustomDataset(Dataset):
    """Landsat Dataset images"""
    
    def __init__(self, dataroot, transform=None):
        """
        Args:
            dataroot (string): Path to images
            transform (callable, optional): Optional transform to be
            appied on a sample
        """
        self.dataroot = dataroot
        self.transform = transform
        self.image_name = os.listdir(self.dataroot)
    
    def __len__(self):
        """Get size of dataset"""
        
        return len(self.image_name)
    
    def __getitem__(self, idx):
        """Link for images in dataset and transforming if also need"""
        
        image_path = os.path.join(self.dataroot,self.image_name[idx])
        image = Image.open(image_path)
        if self.transform:
            image = self.transform(image)
        return image

In [ ]:
def show_tensor_images(image_tensor:torch.Tensor, num_images:int, size:tuple):
    """
    Function for visual tensor
    Args:
        image_tensor(Tensor): image tensor
        num_images(int): quntity image on grid
        size(tuple(channel,h,w)): size per image 
    """

    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [ ]:
class Generator(nn.Module):
    """
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_chan: the number of channels in the images
        hidden_dim: the inner dimension, a scalar
    """

    def __init__(self, z_dim=100, im_chan=3, hidden_dim=64):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        # Build the neural network
        self.gen = nn.Sequential(
            self.make_gen_block(z_dim, hidden_dim * 8, kernel_size=4, stride=1),
            self.make_gen_block(hidden_dim * 8, hidden_dim * 4),
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2),
            self.make_gen_block(hidden_dim * 2, hidden_dim),
            self.make_gen_block(hidden_dim, hidden_dim, kernel_size=3, stride=1),
            self.make_gen_block(hidden_dim, im_chan, kernel_size=4, final_layer=True),
        )

    def make_gen_block(self, input_channels, output_channels, kernel_size=4, stride=2, padding=1, final_layer=False):
        '''
        Function to return a sequence of operations corresponding to a generator block of DCGAN, 
        corresponding to a transposed convolution, a batchnorm (except for in the last layer), and an activation.
        Parameters:
            input_channels: how many channels the input feature representation has
            output_channels: how many channels the output feature representation should have
            kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false otherwise 
                      (affects activation and batchnorm)
        '''

        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride,padding,bias=False),
                nn.ReLU(True),
                nn.BatchNorm2d(output_channels)
            )
        else: # Final Layer
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride,padding,bias=False),
                nn.Tanh()
            )

    def unsqueeze_noise(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns a copy of that noise with width and height = 1 and channels = z_dim.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        return noise.view(len(noise), self.z_dim, 1, 1)

    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor, 
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        x = self.unsqueeze_noise(noise)
        return self.gen(x)

def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim)
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
        n_samples: the number of samples to generate, a scalar
        z_dim: the dimension of the noise vector, a scalar
        device: the device type
    '''
    return torch.randn(n_samples, z_dim, device=device)

In [ ]:
class Discriminator(nn.Module):
    '''
    Discriminator Class
    Values:
        im_chan: the number of channels in the images.
    hidden_dim: the inner dimension, a scalar
    '''

    def __init__(self, im_chan=3, hidden_dim=64):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            self.make_disc_block(im_chan, hidden_dim, first_layer=True),
            self.make_disc_block(hidden_dim, hidden_dim, kernel_size=3, stride=1),
            self.make_disc_block(hidden_dim, hidden_dim * 2),
            self.make_disc_block(hidden_dim * 2, hidden_dim * 4),
            self.make_disc_block(hidden_dim * 4, hidden_dim * 8),
            self.make_disc_block(hidden_dim * 8, 1, stride=1, final_layer=True),
        )

    def make_disc_block(self, input_channels, output_channels, kernel_size=4, stride=2,padding=1, final_layer=False, first_layer=False):
        '''
        Function to return a sequence of operations corresponding to a discriminator block of DCGAN, 
        corresponding to a convolution, a batchnorm (except for in the last layer), and an activation.
        Parameters:
            input_channels: how many channels the input feature representation has
            output_channels: how many channels the output feature representation should have
            kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false otherwise 
                      (affects activation and batchnorm)
        '''

        if not (final_layer or first_layer):
            return nn.Sequential(
                nn.Conv2d(input_channels,output_channels,kernel_size,stride,padding,bias=False),
                nn.LeakyReLU(0.2,True),
                nn.BatchNorm2d(output_channels)
                
            )
        elif first_layer:
            return nn.Sequential(
                nn.Conv2d(input_channels,output_channels,kernel_size,stride,padding,bias=False),
                nn.LeakyReLU(0.2,True)
            )
        else: # Final Layer
            return nn.Sequential(
                nn.Conv2d(input_channels,output_channels,kernel_size,stride,padding,bias=False),
                nn.LeakyReLU(0.2,True)
            )

    def forward(self, image):
        '''
        Function for completing a forward pass of the discriminator: Given an image tensor, 
        returns a 1-dimension tensor representing fake/real.
        Parameters:
            image: a flattened image tensor with dimension (im_dim)
        '''
        disc_pred = self.disc(image)
        return disc_pred.view(len(disc_pred), -1)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
z_dim = 100

lr = 2e-4
batch_size = 1024
beta_1 = 0.5 
beta_2 = 0.999
device = 'cuda'

In [ ]:
gen = Generator(z_dim).to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr=lr, betas=(beta_1, beta_2))
disc = Discriminator().to(device) 
disc_opt = torch.optim.Adam(disc.parameters(), lr=lr, betas=(beta_1, beta_2))

def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)
gen = gen.apply(weights_init)
disc = disc.apply(weights_init)

In [ ]:
print(gen)

In [ ]:
print(disc)

In [ ]:
torch.cuda.synchronize()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)),
])
dataroot = "/content/drive/MyDrive/datasets/gee/dataset/tibet_200k/TIBET"
dataset = CustomDataset(dataroot,transform=transform)
dataloader = DataLoader(dataset,batch_size, pin_memory=True, shuffle=True)

In [ ]:
len_dataset = len(dataset)

In [ ]:
display_step = len_dataset//batch_size
print(f"batch size: {batch_size}, dataset size: {len_dataset}, display_step: {display_step}")

In [ ]:
n_epochs = 500
cur_step = 0
mean_generator_loss = 0
mean_discriminator_loss = 0
header = True
for epoch in range(n_epochs):
    # Dataloader returns the batches
    for real in tqdm(dataloader):
        cur_batch_size = len(real)
        real = real.to(device)

        ## Update discriminator ##
        disc_opt.zero_grad()
        fake_noise = get_noise(cur_batch_size, z_dim, device=device)
        fake = gen(fake_noise)
        disc_fake_pred = disc(fake.detach())
        disc_fake_loss = criterion(disc_fake_pred, torch.zeros_like(disc_fake_pred))
        disc_real_pred = disc(real)
        disc_real_loss = criterion(disc_real_pred, torch.ones_like(disc_real_pred))
        disc_loss = (disc_fake_loss + disc_real_loss) / 2

        mean_discriminator_loss += disc_loss.item() / display_step
        # Update gradients
        disc_loss.backward(retain_graph=True)
        # Update optimizer
        disc_opt.step()

        ## Update generator ##
        gen_opt.zero_grad()
        fake_noise_2 = get_noise(cur_batch_size, z_dim, device=device)
        fake_2 = gen(fake_noise_2)
        disc_fake_pred = disc(fake_2)
        gen_loss = criterion(disc_fake_pred, torch.ones_like(disc_fake_pred))
        gen_loss.backward()
        gen_opt.step()

        mean_generator_loss += gen_loss.item() / display_step

        # Visual
        if cur_step % display_step == 0 and cur_step > 0:
            print(f"Step {cur_step}: Generator loss: {mean_generator_loss}, discriminator loss: {mean_discriminator_loss}")
            show_tensor_images(fake, num_images=25, size=(3,64,64))
            show_tensor_images(real, num_images=25, size=(3,64,64))
            save_image(make_grid(fake[:16]), f"/content/drive/MyDrive/research-work/output_dcgan/{epoch}_{cur_step}_{str(datetime.now())}.png")
            with open("report-dcgan.csv", "a") as f:
                writer = csv.DictWriter(f,fieldnames=["epoch","cur_step","loss","model"])
                if header:
                    writer.writeheader()
                    writer.writerow({"epoch":epoch,"cur_step":cur_step,"loss":mean_generator_loss,"model":"generator"})    
                    writer.writerow({"epoch":epoch,"cur_step":cur_step,"loss":mean_discriminator_loss,"model":"discriminator"})
                    header = False
                else:
                    writer.writerow({"epoch":epoch,"cur_step":cur_step,"loss":mean_generator_loss,"model":"generator"})    
                    writer.writerow({"epoch":epoch,"cur_step":cur_step,"loss":mean_discriminator_loss,"model":"discriminator"})
            mean_generator_loss = 0
            mean_discriminator_loss = 0
        cur_step += 1

In [ ]:
import pandas as pd
import altair as alt

In [ ]:
df = pd.read_csv("/content/report-dcgan.csv")

In [ ]:
df.info()

In [ ]:
chart = alt.Chart(df).mark_line().encode(
    x = "epoch",
    y = "loss",
    color = "model"
)
chart